# Natural Language Processing - Hands-on

Uma das questões mais importantes que os cientistas de dados encontram é como representar seus dados para um algoritmo. Isso é especialmente relevante em NLP, onde as entradas geralmente diferem em comprimento, tomando a forma de sentenças ou mesmo documentos inteiros. Independentemente do tamanho da entrada, é importante desenvolver uma representação que possa capturar temas semelhantes e / ou usos de termos e vocabulário específicos do domínio. Vamos demonstrar isso usando alguns exemplos didáticos e simples para facilitar o entendimento.

## <center>BASE</center>

In [1]:
!pip install transformers

In [2]:
!pip install pyarrow>=5.0.0

In [3]:
!pip install sklearn

In [4]:
!pip install datasets

In [5]:
!pip install datasets

In [6]:
!pip install nltk

  Using cached nltk-3.7-py3-none-any.whl (1.5 MB)


In [7]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2,3,5,6,7,11,12,13' # fará uso da GPU 1 e 2.

In [8]:
#from google.colab import drive
#drive.mount('/content/drive')

#caminho_base = "/content/drive/My Drive/Mestrado/"
caminho_base = "./result"

### Carregando dados 

In [9]:
import pandas as pd


#df_noticias = pd.read_csv(caminho_base + 'noticias.csv')
#df_test = pd.read_csv(caminho_base + 'noticias_test.csv')
#df_classes = pd.read_csv(caminho_base + 'classes.csv', sep=';')

#df_train = df_noticias.merge(df_test['arquivo'], how='left', on='arquivo', indicator=True)
#df_train = df_train[df_train['_merge']!='both']
#df_train = df_train[list(df_noticias.columns)]
#del df_noticias

#df_test['conteudo_tratado_lower'] = df_test['conteudo_tratado'].str.lower()
#df_train['conteudo_tratado_lower'] = df_train['conteudo_tratado'].str.lower()

#df_train = pd.read_csv('C:/Users/Ricardo/cimatec/tcc/Final/final_v2/df_train.csv')
#df_test = pd.read_csv('C:/Users/Ricardo/cimatec/tcc/Final/final_v2/df_test.csv')

df = pd.read_csv('texto_birads_variaveis_birads_nao_balanceado_one_hot_encoding.csv')

In [10]:
df.head()

,Unnamed: 0,texto_do_laudo_3,birads,Category_Code,birads0,birads1,birads2,birads3,birads4,birads4a,birads4c,birads5,birads6
0,56503,ressonancia magnetica mamaria indicacao clini...,categoria 2,2,0,0,1,0,0,0,0,0,0
1,56504,ressonancia magnetica mamaria indicacao clini...,categoria 3,3,0,0,0,1,0,0,0,0,0
2,56505,ressonancia magnetica mamaria indicacao clini...,categoria 3,3,0,0,0,1,0,0,0,0,0
3,56506,ressonancia magnetica mamaria indicacao clinic...,categoria 3,3,0,0,0,1,0,0,0,0,0
4,56507,ressonancia magnetica mamaria indicacao clini...,categoria 3,3,0,0,0,1,0,0,0,0,0


In [11]:
df.rename( columns={'Unnamed: 0':'id'}, inplace=True )

In [12]:
df.drop('birads', axis=1, inplace=True)

In [13]:
df.drop('Category_Code', axis=1, inplace=True)

In [14]:
df.head()

,id,texto_do_laudo_3,birads0,birads1,birads2,birads3,birads4,birads4a,birads4c,birads5,birads6
0,56503,ressonancia magnetica mamaria indicacao clini...,0,0,1,0,0,0,0,0,0
1,56504,ressonancia magnetica mamaria indicacao clini...,0,0,0,1,0,0,0,0,0
2,56505,ressonancia magnetica mamaria indicacao clini...,0,0,0,1,0,0,0,0,0
3,56506,ressonancia magnetica mamaria indicacao clinic...,0,0,0,1,0,0,0,0,0
4,56507,ressonancia magnetica mamaria indicacao clini...,0,0,0,1,0,0,0,0,0


In [15]:
from sklearn.model_selection import train_test_split

In [16]:
df_train, df_test = train_test_split(df, random_state=2020, test_size=0.30)

In [17]:
#from sklearn.feature_extraction.text import TfidfVectorizer
#import nltk
#import numpy as np
#from nltk.corpus import stopwords

#tfidf_vectorizer = TfidfVectorizer(
#        analyzer="word", stop_words=stopwords.words("portuguese"),
#        min_df = 0.001, max_df = 0.999
#    )
#tfidf_vectorizer.fit_transform(df_train.conteudo_tratado)
#new_tokens = list(tfidf_vectorizer.vocabulary_.keys())

In [18]:
from datasets import Dataset

df = pd.DataFrame(df_train.texto_do_laudo_3.values, columns=['text'])

dataset_train = Dataset.from_pandas(df)

df = pd.DataFrame(df_test.texto_do_laudo_3.values, columns=['text'])

dataset_test = Dataset.from_pandas(df)

In [19]:
import multiprocessing

jobs = 50 #multiprocessing.cpu_count() - 5

from transformers import BertTokenizer, DataCollatorForLanguageModeling, BertForMaskedLM
from tokenizers  import BertWordPieceTokenizer
import numpy as np
#tokenizer = BertTokenizer.from_pretrained('/home/raphael.oliveira/dados/BERT', do_lower_case=False, strip_accents=False)
tokenizerOriginal = BertTokenizer.from_pretrained('neuralmind/bert-large-portuguese-cased', do_lower_case=False, strip_accents=False)
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-large-portuguese-cased', do_lower_case=False, strip_accents=False)

#model = AutoModelForMaskedLM.from_pretrained('/home/raphael.oliveira/dados/BERT')
model = BertForMaskedLM.from_pretrained('neuralmind/bert-large-portuguese-cased')
model.train()

actual_vocab_file = 'vocab.txt'

### add new tokens to the existing vocabulary (only those not already presents)
#print("[ BEFORE ] tokenizer vocab size:", len(tokenizer)) 
#added_tokens = tokenizer.add_tokens(new_tokens)
#print("[ AFTER ] tokenizer vocab size:", len(tokenizer)) 
#print()
#print('added_tokens:',added_tokens)
#print()# resize the embeddings matrix of the model 
#model.resize_token_embeddings(len(tokenizer))

Some weights of the model checkpoint at neuralmind/bert-large-portuguese-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
from pathlib import Path
import shutil
"""Adds new vocab to tokenizer and randomly initialises rows for new vocab in the model"""
PATH = Path('./lm_data')
PATH.mkdir(exist_ok=True)

lm_text = ' '.join(df['text'])
#unwanted_char = '\x91\x92\x93\x94\x95\x96'
#lm_text = "".join([(" " if n in unwanted_char else n) for n in lm_text if n not in unwanted_char])

#lm_text = lm_text.replace("■","")
#lm_text = re.sub(r'[^\x00-\x7F]+',' ', lm_text)
#lm_text = re.sub(r'[^\u0000-\u007F]+',' ', lm_text)

#from sklearn.feature_extraction.text import TfidfVectorizer
#import nltk
#import numpy as np
#from nltk.corpus import stopwords

#tfidf_vectorizer = TfidfVectorizer(
#        analyzer="word", stop_words=stopwords.words("portuguese"),
#        min_df = 0.001, max_df = 0.999
#    )
#tfidf_vectorizer.fit_transform(df_train.conteudo_tratado_lower)
#new_tokens = list(tfidf_vectorizer.vocabulary_.keys())
#lm_text = ' '.join(new_tokens)

with open('./lm_data/data.txt', mode='w') as f:
    f.write(lm_text)

paths = [str(x) for x in PATH.glob("**/*.txt")]
#print(paths)

vocab = list(tokenizer.get_vocab().keys())
#print(len(vocab))
# Initialize a tokenizer
tokenizer_new = BertWordPieceTokenizer(actual_vocab_file, lowercase=True, strip_accents=False)

# Customize training
tokenizer_new.train(files=paths, 
                    #vocab_size=vocab_size, 
                    min_frequency=5, 
                    show_progress=True, 
                    special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],
                    limit_alphabet=1000
                   )
new_vocab = list(tokenizer_new.get_vocab().keys())    
new_vocab = list(set(new_vocab)-set(vocab))
print(len(new_vocab))

print('Adding new tokens to vocab')
n_orig_tokens = len(tokenizer)
#add = tokenizer.add_tokens(new_tokens)
#print('Original no. of tokens: %s'%n_orig_tokens)
#print(add)    
add = tokenizer.add_tokens(new_vocab)
print('Original no. of tokens: %s'%n_orig_tokens)
print(add)
print('Final no. of tokens: %s'%len(tokenizer))

print('Initialised random emb for new tokens')
model.resize_token_embeddings(len(tokenizer))

shutil.rmtree(PATH)




1819
Adding new tokens to vocab
Original no. of tokens: 29794
1819
Final no. of tokens: 31613
Initialised random emb for new tokens


In [ ]:
#tokenizer.save_pretrained(caminho_base + 'BERT/tokenizer')

In [21]:
from datasets import Dataset
from datasets import logging

In [22]:
from transformers import DataCollatorForLanguageModeling
from datasets import logging
import nltk
import numpy as np
logging.set_verbosity_info()

#def tokenize_function(examples):
#    return tokenizer(examples["text"])
def convert_tokens_to_ids(text, unk_id, cls_id, sep_id):
    #print(text)    
    doc_ids = [cls_id]
    for token in nltk.word_tokenize(text):
        id = tokenizer.convert_tokens_to_ids(token)
        if (id == unk_id):        
            ids = [tokenizerOriginal.convert_tokens_to_ids(t) for t in tokenizerOriginal.tokenize(token)]                    
        else:
            ids = [id]
        if (len(ids) > 0):
            doc_ids.extend(ids)
    doc_ids.append(sep_id)
    return doc_ids

def doc_tokenizer(docs):
    unk_id = tokenizerOriginal.convert_tokens_to_ids(tokenizerOriginal.unk_token) 
    cls_id = tokenizerOriginal.convert_tokens_to_ids(tokenizerOriginal.cls_token)
    sep_id = tokenizerOriginal.convert_tokens_to_ids(tokenizerOriginal.sep_token)
    input_ids = []
    token_type_ids = []
    attention_mask = []
    special_tokens_mask = []
    for text in docs['text']:        
        text_id = convert_tokens_to_ids(text, unk_id, cls_id, sep_id)
        input_ids.append(text_id)
        token_type_ids.append(np.zeros(len(text_id), dtype=int))
        attention_mask.append(np.ones(len(text_id), dtype=int))
        stm = []
        stm.append(1)
        stm.extend(np.zeros(len(text_id) - 2, dtype=int))
        stm.append(1)
        special_tokens_mask.append(stm)
    
    return {'input_ids': input_ids, 'token_type_ids': token_type_ids, 'attention_mask': attention_mask, 'special_tokens_mask': special_tokens_mask}


block_size = 64
back_size = 16
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    size = len(concatenated_examples[list(examples.keys())[0]])    
    slices = []        
    start = 0
    end = min(block_size, size)
    slices.append(slice(start, end))
    while (end < size):
        start = start + block_size - back_size
        end = start + block_size
        if (end > size):
            end = size
            start = end - block_size        

        slices.append(slice(start, end))
    
    # Split by chunks of max_len.
    result = {
        k: [t[sl] for sl in slices]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.25)

In [23]:
tokenized_dataset_train = dataset_train.map(doc_tokenizer, batched=True, num_proc=jobs, remove_columns=["text"], 
                                            keep_in_memory =True)

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#4:   0%|          | 0/1 [00:00<?, ?ba/s]

...

#44:   0%|          | 0/1 [00:00<?, ?ba/s]

#46:   0%|          | 0/1 [00:00<?, ?ba/s]

#48:   0%|          | 0/1 [00:00<?, ?ba/s]

#47:   0%|          | 0/1 [00:00<?, ?ba/s]

#49:   0%|          | 0/1 [00:00<?, ?ba/s]

In [24]:
tokenized_dataset_test = dataset_test.map(doc_tokenizer, batched=True, num_proc=jobs, remove_columns=["text"], 
                                            keep_in_memory =True)

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#4:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

...

#45:   0%|          | 0/1 [00:00<?, ?ba/s]

#48:   0%|          | 0/1 [00:00<?, ?ba/s]

#43:   0%|          | 0/1 [00:00<?, ?ba/s]

#44:   0%|          | 0/1 [00:00<?, ?ba/s]

#47:   0%|          | 0/1 [00:00<?, ?ba/s]

In [25]:
lm_tokenized_dataset_train = tokenized_dataset_train.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=jobs,
)

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#4:   0%|          | 0/1 [00:00<?, ?ba/s]

...

#45:   0%|          | 0/1 [00:00<?, ?ba/s]

#46:   0%|          | 0/1 [00:00<?, ?ba/s]

#47:   0%|          | 0/1 [00:00<?, ?ba/s]

#48:   0%|          | 0/1 [00:00<?, ?ba/s]

#49:   0%|          | 0/1 [00:00<?, ?ba/s]

In [26]:
lm_tokenized_dataset_test = tokenized_dataset_test.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=jobs,
)
#lm_tokenized_dataset_train[0]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#4:   0%|          | 0/1 [00:00<?, ?ba/s]

...

#45:   0%|          | 0/1 [00:00<?, ?ba/s]

#46:   0%|          | 0/1 [00:00<?, ?ba/s]

#47:   0%|          | 0/1 [00:00<?, ?ba/s]

#48:   0%|          | 0/1 [00:00<?, ?ba/s]

#49:   0%|          | 0/1 [00:00<?, ?ba/s]

In [27]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    caminho_base + "BERT",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    save_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_tokenized_dataset_train,
    eval_dataset=lm_tokenized_dataset_test,
    data_collator=data_collator,
)

In [28]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
/srv/conda/envs/saturn/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 22244
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 8343


Epoch,Training Loss,Validation Loss
1,1.123300,1.022229
2,0.850300,0.828296
3,0.820400,0.774098


The following columns in the evaluation set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9512
  Batch size = 8
Saving model checkpoint to ./resultBERT/checkpoint-2781
Configuration saved in ./resultBERT/checkpoint-2781/config.json
Model weights saved in ./resultBERT/checkpoint-2781/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9512
  Batch size = 8
Saving model checkpoint to ./resultBERT/checkpoint-5562
Configuration saved in ./resultBERT/checkpoint-5562/config.json
Model weights saved in ./res

TrainOutput(global_step=8343, training_loss=1.1223221236705265, metrics={'train_runtime': 62820.3345, 'train_samples_per_second': 1.062, 'train_steps_per_second': 0.133, 'total_flos': 7774515501857280.0, 'train_loss': 1.1223221236705265, 'epoch': 3.0})

In [29]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

The following columns in the evaluation set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9512
  Batch size = 8


Perplexity: 2.17


In [30]:
trainer.save_model(caminho_base + 'BERT/model11')

# alternative saving method and folder
#model.save_pretrained(caminho_base + 'BERT/model2')

Saving model checkpoint to ./resultBERT/model11
Configuration saved in ./resultBERT/model11/config.json
Model weights saved in ./resultBERT/model11/pytorch_model.bin


In [31]:
tokenizer.save_pretrained(caminho_base + 'BERT/tokenizer11')
#tokenizer.save_vocabulary(caminho_base + 'BERT/')

tokenizer config file saved in ./resultBERT/tokenizer11/tokenizer_config.json
Special tokens file saved in ./resultBERT/tokenizer11/special_tokens_map.json
added tokens file saved in ./resultBERT/tokenizer11/added_tokens.json


('./resultBERT/tokenizer11/tokenizer_config.json',
 './resultBERT/tokenizer11/special_tokens_map.json',
 './resultBERT/tokenizer11/vocab.txt',
 './resultBERT/tokenizer11/added_tokens.json')